In [1]:
BRANCH = 'r1.13.0'

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.13.0) to /tmp/pip-install-m40oh75q/nemo-toolkit_dc4ac2f47f55476a85af4d6a94a9e037
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-m40oh75q/nemo-toolkit_dc4ac2f47f55476a85af4d6a94a9e037
  Running command git checkout -b r1.13.0 --track origin/r1.13.0
  Switched to a new branch 'r1.13.0'
  Branch 'r1.13.0' set up to track remote branch 'r1.13.0' from 'origin'.
  Resolved https://github.com/NVIDIA/NeMo.git to commit 26e3e1d1931a9f34986f73f071f82f446d90c1c5
  Preparing metadata (setup.py) ... done


In [ ]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

In [3]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp
from nemo.collections import asr as nemo_asr

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[NeMo W 2022-11-03 05:00:48 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-11-03 05:00:48 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-11-03 05:00:49 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


# Task Description
Automatic Speech Recognition (ASR) systems typically generate text with no punctuation and capitalization of the words.
This tutorial explains how to implement a model in NeMo that will predict punctuation and capitalization using both text and audio for each word in a sentence to make ASR output more readable and to boost performance of the named entity recognition, machine translation or text-to-speech models. You can find documentation on text only model [here](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/punctuation_and_capitalization.html).


We'll show how to train a model for this task using a pre-trained [BERT](https://arxiv.org/abs/1810.04805) and [Conformer](https://arxiv.org/abs/2005.08100) models. You can find all pretrained Conformer models on [NGC](https://catalog.ngc.nvidia.com/models?filters=&orderBy=dateModifiedDESC&query=conformer).

For every word in our training dataset we’re going to predict:

- punctuation mark that should follow the word and
- whether the word should be capitalized


In some cases lexical only model can't predict punctuation correctly without audio. It is especially hard for conversational speech.

For example:

- Yeah, over there you walk a lot. -> Yeah, over there you walk a lot?
- Supposedly eighty five percent of your body's liquid, but, you know, just a hassle. -> Supposedly, eighty five percent of your body's liquid. But, you know just a hassle.
- Oh, yeah? -> Oh, yeah.


## Architecture
Punctuation and capitalization lexical audio model is based on [Multimodal Semi-supervised Learning Framework for Punctuation Prediction in Conversational Speech](https://arxiv.org/pdf/2008.00702.pdf). Model consists of lexical encoder (BERT-like model), acoustic encoder (i.e. Conformer's audio encoder), fusion of lexical and audio features (attention based fusion) and prediction layers.

Fusion is needed because encoded text and audio might have different length therefore can't be aligned one-to-one. As model predicts punctuation and capitalization per text token we use cross-attention between encoded lexical and encoded audio input.

# Dataset
This model can work with any dataset as long as it follows the format specified below.
The training and evaluation data is divided into *3 files: text.txt, labels.txt and audio.txt*.
Each line of the **text.txt** file contains text sequences, where words are separated with spaces: [WORD] [SPACE] [WORD] [SPACE] [WORD], for example:



```
when is the next flight to new york
the next flight is ...
...
```


The **labels.txt** file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each label in labels.txt file consists of 2 symbols:

- the first symbol of the label indicates what punctuation mark should follow the word (where O means no punctuation needed);
- the second symbol determines if a word needs to be capitalized or not (where U indicates that the word should be upper cased, and O - no capitalization needed.)

In this tutorial, we are considering only commas, periods, and question marks the rest punctuation marks were removed. To use more punctuation marks, update the dataset to include desired labels, no changes to the model needed.

Each line of the **labels.txt** should follow the format:
[LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt).
For example, labels for the above text.txt file should be:



```
OU OO OO OO OO OO OU ?U
OU OO OO OO ...
...
```


The complete list of all possible labels for this task used in this tutorial is: `OO, ,O, .O, ?O, OU, ,U, .U, ?U.`


The **audio.txt** file contains corresponding audio's filepath for each sample in text.txt

Each line of the **audio.txt** should follow the format:
/Path/to/audio/file.wav

For example, filepaths for the above text.txt should be:



```
/path/1.wav
/path/2.wav
...
```



## Download and preprocess the data

In this notebook we are going to use a subset of [LibriTTS](https://www.openslr.org/60/). This script will download and preprocess the LibriTTS data [NeMo/examples/nlp/token_classification/get_libritts_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_libritts_data.py).
**Note:** for simplicity we will use dev subset as train and dev subsets

In [4]:
DATA_DIR = './data'
WORK_DIR = './scripts'
MODEL_CONFIG = "punctuation_capitalization_lexical_audio_config.yaml"

# model parameters
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002

In [5]:
# download get_libritts_data.py script to download and preprocess the LibriTTS data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_libritts_data.py'):
    print('Downloading get_libritts_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_libritts_data.py', WORK_DIR)
else:
    print ('get_libritts_data.py already exists')

100% [................................................................................] 4804 / 4804

In [8]:
# download and preprocess the data
# we will use dev_clean and dev_other subsets
# --clean flag deletes raw data
!python $WORK_DIR/get_libritts_data.py --data_dir $DATA_DIR --clean --data_set dev_clean,dev_other

Traceback (most recent call last):
  File "/home/mnovikov/NeMo/tutorials/nlp/./scripts/get_libritts_data.py", line 29, in <module>
    from get_tatoeba_data import create_text_and_labels
ModuleNotFoundError: No module named 'get_tatoeba_data'


after execution of the above cell, your data folder will contain the following 3 files needed for training (raw LibriTTS data could be present if `--clean` was not used):
- labels_dev.txt
- text_dev.txt
- audio_dev.txt

In [ ]:
!ls -l $DATA_DIR

In [ ]:
# let's take a look at the data
print('Raw text:')
!head -n 5 $DATA_DIR/text_dev.txt

print('\nLabels:')
!head -n 5 $DATA_DIR/labels_dev.txt

print('\nFilepaths: ')
!head -n 5 $DATA_DIR/audio_dev.txt

As you see, `get_libritts_data.py` script provides not only downloads LibriTTS but also creates labels. If you wish to preprocess your own data, use [examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py) script.

# Tarred dataset

If your dataset is too large to be stored in memory, you can use tarred dataset. A tarred dataset is a collection of tarred files which contain batches ready for passing into a model.

All tar files will contain identical number of batches, so if number of batches in the dataset is not evenly divisible by parameter `--num_batches_per_tar_file` value, then up to `--num_batches_per_tar_file - 1` batches may be lost. More details on [tarred dataset](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/punctuation_and_capitalization.html#tarred-dataset) and details specific to lexical and audio model's [tarred dataset](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/punctuation_and_capitalization_lexical_audio.html#tarred-dataset).

In [ ]:
# Number of lines in text and labels files
!wc -l $DATA_DIR/text_dev.txt
!wc -l $DATA_DIR/labels_dev.txt

In [ ]:
## download script to prepare tarred dataset
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(f"{WORK_DIR}/create_punctuation_capitalization_tarred_dataset.py"):
    print('Downloading create_punctuation_capitalization_tarred_dataset.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py', WORK_DIR)
else:
    print ("create_punctuation_capitalization_tarred_dataset.py script already exists")

!python $WORK_DIR/create_punctuation_capitalization_tarred_dataset.py \
    --text $DATA_DIR/text_dev.txt \
    --labels $DATA_DIR/labels_dev.txt \
    --output_dir $DATA_DIR/train_tarred \
    --num_batches_per_tarfile 20 \
    --tokens_in_batch 1024 \
    --lines_per_dataset_fragment 4000 \
    --tokenizer_name bert-base-uncased \
    --n_jobs 2 \
    --use_audio \
    --sample_rate 16000 \
    --audio_file $DATA_DIR/audio_dev.txt


In [ ]:
!ls $DATA_DIR/train_tarred -l

In [ ]:
!ls $DATA_DIR/train_tarred/*.tar | wc -l  # number of tar files

In [ ]:
!ls $DATA_DIR/train_tarred/ | grep -v '.tar'  # all not tar files

If you wish to use tarred dataset, then you need to
- pass metadata JSON file in config parameter `model.train_ds.tar_metadata_file`,
- set `model.train_ds.use_tarred_dataset=true`.

# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the fusion of pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) and encoder of ASR model models:
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

See [docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model) for full config description.

In [ ]:
# download the model's configuration file
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

# Setting up Data within the config

Among other things, the config file contains dictionaries called `common_dataset_parameters`, `train_ds` and `validation_ds`. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

Specify paths directories with train and dev datasets in parameters `train_ds.ds_item` and `validation_ds.ds_item`.

If you want to use multiple datasets for evaluation, specify paths to the directory(ies) with evaluation file(s) in the following way:

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` (Note no space between the paths and square brackets).

Also notice that some configs, including `model.train_ds.ds_item`, have `???` in place of values, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [ ]:
# in this tutorial train and dev data is located in the same folder
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 1

# Remove distributed training flags
config.trainer.strategy = None
config.exp_manager.use_datetime_version=False
config.exp_manager.explicit_log_dir='Punctuation_And_Capitalization_Lexical_Audio'

trainer = pl.Trainer(**config.trainer)

# Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model and Conformer model.

In [ ]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

PRETRAINED_BERT_MODEL = "bert-base-uncased"

# complete list of supported ASR models
print(nemo_asr.models.ASRModel.list_available_models())

PRETRAINED_ASR_MODEL = "stt_en_conformer_ctc_small"

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.train_ds.text_file = 'text_dev.txt'
config.model.train_ds.labels_file = 'labels_dev.txt'
config.model.train_ds.audio_file = 'audio_dev.txt'
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.audio_encoder.pretrained_model = PRETRAINED_ASR_MODEL
config.model.train_ds.preload_audios = True
config.model.validation_ds.preload_audios = True

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [ ]:
# initialize the model
# during this stage, the dataset and data loaders we'll be prepared for training and evaluation
model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel(cfg=config.model, trainer=trainer)

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# start the training
trainer.fit(model)

# Training using tarred dataset

In [ ]:
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = f'{DATA_DIR}/train_tarred'
config.model.train_ds.use_tarred_dataset = True
# Only metadata file name is required if `use_tarred_dataset=true`.
config.model.train_ds.tar_metadata_file = 'metadata.punctuation_capitalization.tokens1024.max_seq_length512.bert-base-uncased.json'
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Trainer
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.max_epochs = 1
config.trainer.strategy = None

# Exp manager
config.exp_manager.explicit_log_dir = 'tarred_experiment'

config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.validation_ds.preload_audios = True
config.model.audio_encoder.pretrained_model = PRETRAINED_ASR_MODEL

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_manager.use_datetime_version=False
exp_manager.explicit_log_dir='Punctuation_And_Capitalization_Lexical_Audio'
model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel(cfg=config.model, trainer=trainer)
trainer.fit(model)

# Inference using a pretrained model

For [inference](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/punctuation_and_capitalization_lexical_audio.html#inference) you can use same script as ``PunctuationCapitalizationModel`` uses, just add **--use_audio** and **--audio_file** parameters.

## Training Script

If you have NeMo installed locally, you can also train the model with [nlp/token_classification/punctuation_capitalization_lexical_audio_train_evaluate.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/punctuation_capitalization_lexical_audio_train_evaluate.py).

To run training script, use:

`python punctuation_capitalization_lexical_audio_train_evaluate.py model.train_ds.ds_item=PATH_TO_TRAIN_DATA_DIR`


# Finetuning model with your data

When we were training the model from scratch, the datasets were prepared for training during the model initialization. When we are using a pretrained Punctuation and Capitalization model, before training, we need to setup training and evaluation data.

In [ ]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel.from_pretrained('Punctuation_And_Capitalization_Lexical_Audio/checkpoints/Punctuation_and_Capitalization_Lexical_Audio.nemo')

# setup train and validation Pytorch DataLoaders
pretrained_model.update_config_after_restoring_from_checkpoint(
    train_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_train.txt',
        'labels_file': 'labels_train.txt',
        'tokens_in_batch': 1024,
    },
    validation_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_dev.txt',
        'labels_file': 'labels_dev.txt',
        'tokens_in_batch': 1024,
    },
)

# and now we can create a PyTorch Lightning trainer and call `fit` again
# for this tutorial we are setting fast_dev_run to True, and the trainer will run 1 training batch and 1 validation batch
# for actual model training, disable the flag
fast_dev_run = True
trainer = pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
pretrained_model.set_trainer(trainer)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()
trainer.fit(pretrained_model)

## Next steps

In this tutorial we used fairly small amount of data for showcase purposes. If you wish to train your own model you probably need to collect more data.

For more details on model you can read [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/punctuation_and_capitalization_lexical_audio.html#quick-start-guide). 
